# SQLite and Python

The first thing I did for this homework was to download SQlite browser (https://sqlitebrowser.org/blog/version-3-11-2-released/) for me just to be sure it is there. Then, I created functions, which helped me to connect and load data to SQLite and also query tables after data is loaded.

In [5]:
import sqlite3

# create connection to SQLite
def load_data_to_SQLite(db_name, table_name, column_name1, column_name2):
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    cur.execute(f"DROP TABLE IF EXISTS {table_name};")
    cur.execute(f"CREATE TABLE {table_name} ({column_name1} TEXT, {column_name2} REAL);")
    cur.executemany(f"INSERT INTO {table_name} ({column_name1}, {column_name2}) VALUES (?, ?);", to_db)
    con.commit()
    con.close()

# querying in SQLite
def load_data_from_SQLite(db_name, query):
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    result = cur.execute(query)
    for i, row in enumerate(result, start=1):
        print(i, row)
    con.close()
    return result

In this part, I use my load_data_to_SQlite function to load staty_lide.log and staty_plochy.log into SQLite.

In [6]:
# load data into python IDE as dictionary
country_area = {}
with open('staty_plochy.log', 'r') as file:
    for row in file:
        country, area = row.strip().split(': ')
        country_area[country] = float(area)
        to_db = [(key, val) for key, val in country_area.items()]
        # loading data to SQLite
        load_data_to_SQLite('staty.db', 'staty', 'Countries', 'Areas') # staty.db does not exist so will be created

# load data into python IDE as dictionary
country_people = {}
with open('staty_lidi.log', 'r') as file:
    for row in file:
        country, people = row.strip().split(': ')
        country_people[country] = int(people)
        to_db = [(key, val) for key, val in country_people.items()]
        # loading data to SQLite
        load_data_to_SQLite('staty.db', 'lide', 'Countries', 'Population')  # data will be loaded to staty.db as it exists

In this part I answer the questions which are stated in the homework.

In [7]:
# the ten most populous states in the world, including their area
print('The ten most populous states in the world, including their area are: ')
load_data_from_SQLite('staty.db',
                    '''SELECT lide.Countries, lide.Population, staty.Areas
                        FROM lide INNER JOIN staty ON staty.Countries = lide.Countries
                        ORDER BY lide.Population DESC
                        LIMIT 10''')
print()

# the ten most densely populated countries
print('The ten most densely populated countries are: ')
load_data_from_SQLite('staty.db',
                    '''SELECT lide.Countries, lide.Population/staty.Areas AS density
                        FROM lide INNER JOIN staty ON staty.Countries = lide.Countries
                        ORDER BY density DESC
                        LIMIT 10''')
print()

# countries where nobody lives
print('Countries where nobody lives are: ')
load_data_from_SQLite('staty.db',
                    '''SELECT staty.Countries
                        FROM staty LEFT OUTER JOIN lide ON staty.Countries = lide.Countries
                        WHERE lide.Population IS NULL''')

The ten most populous states in the world, including their area are: 
1 ('China', 1300000000.0, 9596960.0)
2 ('India', 1070000000.0, 3287590.0)
3 ('United States', 293000000.0, 9631418.0)
4 ('Indonesia', 238000000.0, 1919440.0)
5 ('Brazil', 184000000.0, 8511965.0)
6 ('Pakistan', 159000000.0, 803940.0)
7 ('Russia', 144000000.0, 17075200.0)
8 ('Bangladesh', 141000000.0, 144000.0)
9 ('Nigeria', 137000000.0, 923768.0)
10 ('Japan', 127000000.0, 377835.0)

The ten most densely populated countries are: 
1 ('Macau', 17519.68503937008)
2 ('Monaco', 16564.102564102566)
3 ('Hong Kong', 6282.051282051282)
4 ('Singapore', 6279.774794283239)
5 ('Gibraltar', 4276.923076923077)
6 ('Gaza Strip', 3666.6666666666665)
7 ('Holy See (Vatican City)', 2093.181818181818)
8 ('Malta', 1256.3291139240507)
9 ('Bermuda', 1217.6360225140713)
10 ('Maldives', 1130.0)

Countries where nobody lives are: 
1 ('French Southern and Antarctic Lands',)
2 ('South Georgia and the South Sandwich Islands',)
3 ('Jan Mayen',)
4 ('D